In [24]:
# %pip install keras==3.0.2

In [25]:
import random
import string
import tensorflow as tf
import keras
print(tf.__version__)
print(keras.__version__)

from keras import optimizers
from keras import layers
from keras.layers import TextVectorization
import keras.ops as ops

import tensorflow.data as tf_data
import tensorflow.strings as tf_strings

import re

2.15.0
3.0.2


## Чистимо текст (видаляємо останню колонку у перекладах)

In [26]:
def CleanAllText(filePath: str):
    with open(filePath, 'r+', encoding='UTF8') as file:
        lines = file.readlines()
        for i, line in enumerate(lines):
            columns = line.split('\t')
            updated_line = '\t'.join(columns[:-1]) + '\n'
            lines[i] = updated_line
        file.seek(0)
        file.writelines(lines)
        file.truncate()

In [27]:
CleanAllText('slk.txt')

## Розділяємо перекладений текст на 2 колонки (а саме розбиття на пари текстів та додавання токенів [start] + [end])

In [28]:
def ReadAllText(filePath: str):
    with open(filePath, encoding='UTF8') as f:
        lines = f.read().split("\n")[:-1]
    text_pairs = []
    for line in lines:
        eng, slk = line.split("\t")
        slk = "[start] " + slk + " [end]"
        text_pairs.append((eng, slk))
    return text_pairs

In [29]:
text_pairs = ReadAllText('slk.txt')
for i in range(10):
    print(text_pairs[i])

('Go.', '[start] Choď! [end]')
('Go.', '[start] Choďte! [end]')
('Go.', '[start] Iď! [end]')
('Go.', '[start] Iďte! [end]')
('Hi.', '[start] Zdravím! [end]')
('Hi.', '[start] Zdravíčko! [end]')
('Hi.', '[start] Ahojky! [end]')
('Run!', '[start] Utekaj! [end]')
('Run!', '[start] Bež! [end]')
('Run!', '[start] Bežte! [end]')


### Машинний переклад на трансформерах
(ініціалізуємо портібні функції та конфігуруємо модель)

In [30]:
random.shuffle(text_pairs)
num_val_samples = int(0.2 * len(text_pairs))
num_train_samples = len(text_pairs) - 2 * num_val_samples
train_pairs = text_pairs[:num_train_samples]
val_pairs = text_pairs[num_train_samples : num_train_samples + num_val_samples]
test_pairs = text_pairs[num_train_samples + num_val_samples :]

print(f"{len(text_pairs)} total pairs")
print(f"{len(train_pairs)} training pairs")
print(f"{len(val_pairs)} validation pairs")
print(f"{len(test_pairs)} test pairs")

11754 total pairs
7054 training pairs
2350 validation pairs
2350 test pairs


In [31]:
strip_chars = string.punctuation
strip_chars = strip_chars.replace("[", "")
strip_chars = strip_chars.replace("]", "")

vocab_size = 10000
sequence_length = 15
batch_size = 128

In [32]:
def custom_standardization(input_string):
    lowercase = tf_strings.lower(input_string)
    return tf_strings.regex_replace(lowercase, "[%s]" % re.escape(strip_chars), "")

In [33]:
eng_vectorization = TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length,
)

In [34]:
slk_vectorization = TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length + 1,
    standardize=custom_standardization,
)

In [35]:
train_eng_texts = [pair[0] for pair in train_pairs]
train_slk_texts = [pair[1] for pair in train_pairs]
eng_vectorization.adapt(train_eng_texts)
slk_vectorization.adapt(train_slk_texts)

In [39]:
def format_dataset(eng, slk):
    eng = eng_vectorization(eng)
    slk = slk_vectorization(slk)
    return (
        {
            "encoder_inputs": eng,
            "decoder_inputs": slk[:, :-1],
        },
        slk[:, 1:],
    )

In [40]:
def make_dataset(pairs):
    eng_texts, slk_texts = zip(*pairs)
    eng_texts = list(eng_texts)
    slk_texts = list(slk_texts)
    dataset = tf_data.Dataset.from_tensor_slices((eng_texts, slk_texts))
    dataset = dataset.batch(batch_size)
    dataset = dataset.map(format_dataset)
    return dataset.cache().shuffle(2048).prefetch(16)

In [41]:
train_ds = make_dataset(train_pairs)
val_ds = make_dataset(val_pairs)

In [42]:
for inputs, targets in train_ds.take(1):
    print(f'inputs["encoder_inputs"].shape: {inputs["encoder_inputs"].shape}')
    print(f'inputs["decoder_inputs"].shape: {inputs["decoder_inputs"].shape}')
    print(f"targets.shape: {targets.shape}")

inputs["encoder_inputs"].shape: (128, 15)
inputs["decoder_inputs"].shape: (128, 15)
targets.shape: (128, 15)


In [43]:
class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim
        )
        self.dense_proj = keras.Sequential(
            [
                layers.Dense(dense_dim, activation="relu"),
                layers.Dense(embed_dim),
            ]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.supports_masking = True

    def call(self, inputs, mask=None):
        if mask is not None:
            padding_mask = ops.cast(mask[:, None, :], dtype="int32")
        else:
            padding_mask = None

        attention_output = self.attention(
            query=inputs, value=inputs, key=inputs, attention_mask=padding_mask
        )
        proj_input = self.layernorm_1(inputs + attention_output)
        proj_output = self.dense_proj(proj_input)
        return self.layernorm_2(proj_input + proj_output)

    def get_config(self):
        config = super().get_config()
        config.update(
            {
                "embed_dim": self.embed_dim,
                "dense_dim": self.dense_dim,
                "num_heads": self.num_heads,
            }
        )
        return config

In [44]:
class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, vocab_size, embed_dim, **kwargs):
        super().__init__(**kwargs)
        self.token_embeddings = layers.Embedding(
            input_dim=vocab_size, output_dim=embed_dim
        )
        self.position_embeddings = layers.Embedding(
            input_dim=sequence_length, output_dim=embed_dim
        )
        self.sequence_length = sequence_length
        self.vocab_size = vocab_size
        self.embed_dim = embed_dim

    def call(self, inputs):
        length = ops.shape(inputs)[-1]
        positions = ops.arange(0, length, 1)
        embedded_tokens = self.token_embeddings(inputs)
        embedded_positions = self.position_embeddings(positions)
        return embedded_tokens + embedded_positions

    def compute_mask(self, inputs, mask=None):
        if mask is None:
            return None
        else:
            return ops.not_equal(inputs, 0)

    def get_config(self):
        config = super().get_config()
        config.update(
            {
                "sequence_length": self.sequence_length,
                "vocab_size": self.vocab_size,
                "embed_dim": self.embed_dim,
            }
        )
        return config

In [45]:
class TransformerDecoder(layers.Layer):
    def __init__(self, embed_dim, latent_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.latent_dim = latent_dim
        self.num_heads = num_heads
        self.attention_1 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim
        )
        self.attention_2 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim
        )
        self.dense_proj = keras.Sequential(
            [
                layers.Dense(latent_dim, activation="relu"),
                layers.Dense(embed_dim),
            ]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.layernorm_3 = layers.LayerNormalization()
        self.supports_masking = True

    def call(self, inputs, encoder_outputs, mask=None):
        causal_mask = self.get_causal_attention_mask(inputs)
        if mask is not None:
            padding_mask = ops.cast(mask[:, None, :], dtype="int32")
            padding_mask = ops.minimum(padding_mask, causal_mask)
        else:
            padding_mask = None

        attention_output_1 = self.attention_1(
            query=inputs, value=inputs, key=inputs, attention_mask=causal_mask
        )
        out_1 = self.layernorm_1(inputs + attention_output_1)

        attention_output_2 = self.attention_2(
            query=out_1,
            value=encoder_outputs,
            key=encoder_outputs,
            attention_mask=padding_mask,
        )
        out_2 = self.layernorm_2(out_1 + attention_output_2)

        proj_output = self.dense_proj(out_2)
        return self.layernorm_3(out_2 + proj_output)

    def get_causal_attention_mask(self, inputs):
        input_shape = ops.shape(inputs)
        batch_size, sequence_length = input_shape[0], input_shape[1]
        i = ops.arange(sequence_length)[:, None]
        j = ops.arange(sequence_length)
        mask = ops.cast(i >= j, dtype="int32")
        mask = ops.reshape(mask, (1, input_shape[1], input_shape[1]))
        mult = ops.concatenate(
            [ops.expand_dims(batch_size, -1), ops.convert_to_tensor([1, 1])],
            axis=0,
        )
        return ops.tile(mask, mult)

    def get_config(self):
        config = super().get_config()
        config.update(
            {
                "embed_dim": self.embed_dim,
                "latent_dim": self.latent_dim,
                "num_heads": self.num_heads,
            }
        )
        return config

In [46]:
embed_dim = 128
latent_dim = 1024
num_heads = 8

encoder_inputs = keras.Input(shape=(None,), dtype="int64", name="encoder_inputs")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(encoder_inputs)
encoder_outputs = TransformerEncoder(embed_dim, latent_dim, num_heads)(x)
encoder = keras.Model(encoder_inputs, encoder_outputs)

decoder_inputs = keras.Input(shape=(None,), dtype="int64", name="decoder_inputs")
encoded_seq_inputs = keras.Input(shape=(None, embed_dim), name="decoder_state_inputs")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(decoder_inputs)
x = TransformerDecoder(embed_dim, latent_dim, num_heads)(x, encoded_seq_inputs)
x = layers.Dropout(0.5)(x)
decoder_outputs = layers.Dense(vocab_size, activation="softmax")(x)
decoder = keras.Model([decoder_inputs, encoded_seq_inputs], decoder_outputs)

decoder_outputs = decoder([decoder_inputs, encoder_outputs])
transformer = keras.Model(
    [encoder_inputs, decoder_inputs], decoder_outputs, name="transformer"
)

### Тренування моделі використовуючи трансформер для обробки тексту
(Ккомпілюємо із вказаними параметрами + навчаємо 1 епоху використовуючи відповідін набори даних)

In [47]:
epochs = 1
transformer.summary()
transformer.compile(
    "rmsprop", loss="sparse_categorical_crossentropy", metrics=["accuracy"]
)
transformer.fit(train_ds, epochs=epochs, validation_data=val_ds)

Model: "transformer"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃    Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ encoder_inputs            │ (None, None)           │          0 │ -                          │
│ (InputLayer)              │                        │            │                            │
├───────────────────────────┼────────────────────────┼────────────┼────────────────────────────┤
│ positional_embedding      │ (None, None, 128)      │  1,281,920 │ encoder_inputs[0][0]       │
│ (PositionalEmbedding)     │                        │            │                            │
├───────────────────────────┼────────────────────────┼────────────┼────────────────────────────┤
│ decoder_inputs            │ (None, None)           │          0 │ -                          │
│ (InputLayer)              │                        │            │                            │
├───────────────────────────┼────────────────────────┼────────────┼────────────────────────────┤
│ transformer_encoder       │ (None, None, 128)      │    791,296 │ positional_embedding[0][0] │
│ (TransformerEncoder)      │                        │            │                            │
├───────────────────────────┼────────────────────────┼────────────┼────────────────────────────┤
│ functional_5 (Functional) │ (None, None, 10000)    │  3,890,960 │ decoder_inputs[0][0],      │
│                           │                        │            │ transformer_encoder[0][0]  │
└───────────────────────────┴────────────────────────┴────────────┴────────────────────────────┘

 Total params: 5,964,176 (22.75 MB)

 Trainable params: 5,964,176 (22.75 MB)

 Non-trainable params: 0 (0.00 B)

56/56 ━━━━━━━━━━━━━━━━━━━━ 218s 4s/step - accuracy: 0.6077 - loss: 6.0296 - val_accuracy: 0.7218 - val_loss: 2.3259


In [51]:
slk_vocab = slk_vectorization.get_vocabulary()
slk_index_lookup = dict(zip(range(len(slk_vocab)), slk_vocab))
max_decoded_sentence_length = 20

### Результат

In [52]:
def decode_sequence(input_sentence):
    tokenized_input_sentence = eng_vectorization([input_sentence])
    decoded_sentence = "[start]"
    for i in range(max_decoded_sentence_length):
        tokenized_target_sentence = slk_vectorization([decoded_sentence])[:, :-1]
        predictions = transformer([tokenized_input_sentence, tokenized_target_sentence])
        sampled_token_index = ops.convert_to_numpy(
            ops.argmax(predictions[0, i, :])
        ).item(0)
        sampled_token = slk_index_lookup[sampled_token_index]
        decoded_sentence += " " + sampled_token
        if sampled_token == "[end]":
            break
    return decoded_sentence

In [53]:
test_eng_texts = [pair[0] for pair in test_pairs]
for _ in range(30):
    input_sentence = random.choice(test_eng_texts)
    translated = decode_sequence(input_sentence)
    print(input_sentence, translated)

If you plant an apple seed, it might grow into a tree. [start] tom [end]
Tom helped. [start] tom [end]
After lunch, we went shopping. [start] tom [end]
Where will you go for the vacation? [start] tom [end]
A cat was sitting on the chair. [start] tom [end]
The room has two windows. [start] tom [end]
Are you sure you want me to sell this? [start] tom [end]
It was a calm winter evening. [start] tom [end]
What a beautiful garden! [start] tom [end]
Where are you going with this? [start] tom [end]
We really don't have time. [start] tom [end]
It really happened. [start] tom [end]
Tom congratulated Mary. [start] tom [end]
Where's my wife? [start] tom [end]
Do you think Tom is the one who broke the window? [start] tom [end]
The fact that I'm here proves that I'm innocent. [start] tom [end]
Does it surprise you? [start] tom [end]
That wasn't smart. [start] tom [end]
I said what I thought. [start] tom [end]
Tom is unharmed. [start] tom [end]
I have a black dog and a white dog. [start] tom [end]
I

Напревеликий жаль для словацької не вийшло навчити добре модель :(